In [1]:
import sys, os, dotenv
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

In [2]:
from examples._2_with_events.aggregate import Settings, ClassifierSettings
dotenv.load_dotenv()
###
### This is to show  you how a .env file can be used to set the settings
###
settings = Settings(aggregate=ClassifierSettings())
assert settings.aggregate.device == 'cuda'

In [3]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from examples._2_with_events.model import MLP
from examples._2_with_events.dataset import Digits
from examples._2_with_events.repository import Classifiers

# OPTIONAL: Uses the mlregistry library to add a hash and metadata to the model
# I'm gonna use this for storing data in the database
classifiers = Classifiers(settings) 
classifiers.models.register(MLP) 
classifiers.criterions.register(CrossEntropyLoss)
classifiers.optimizers.register(Adam)
classifiers.datasets.register(Digits)

In [4]:
model = MLP(28*28, 128, 10, p=0.5, activation='relu')
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
loaders = [
    ('train', DataLoader(Digits(train=True, normalize=True), batch_size=32, shuffle=True, num_workers=4, pin_memory=True, pin_memory_device='cuda')),
    ('test', DataLoader(Digits(train=False, normalize=True), batch_size=32, shuffle=False, num_workers=4, pin_memory=True, pin_memory_device='cuda'))
]

In [5]:
from torchsystem import Compiler
from torchsystem import Session
from torchsystem.storage import get_metadata

from examples._2_with_events.handlers import subscriber, consumer
from examples._2_with_events.aggregate import Classifier

from logging import getLogger
from logging import basicConfig, INFO
### An example of a complex compilation pipeline
### Note that compiler.compile(classifier) can be used directly
### But this is a way to show how to perform complex compilation steps
### And how you can use it to build your aggregate

compiler = Compiler[Classifier](settings)
logger = getLogger(__name__) #If you don't know what this is, don't worry about it
basicConfig(level=INFO)      #It's just a print

@compiler.step
def build_classifier(model, criterion, optimizer):
    logger.info("Building classifier with:")
    logger.info("Model attributes:")
    logger.info(get_metadata(model))
    logger.info("Criterion attributes:")
    logger.info(get_metadata(criterion))
    logger.info("Optimizer attributes:")
    logger.info(get_metadata(optimizer))
    return Classifier(model, criterion, optimizer, settings)

@compiler.step
def move_to_device(classifier: Classifier):
    return classifier.to(classifier.device)

@compiler.step
def compile_classifier(classifier):
    compiled = compiler.compile(classifier)
    compiled.bind(subscriber, consumer)
    logger.info(f"Compiled classifier on device: {compiled.device}")
    return compiled

In [6]:
from torchsystem.metrics import Callbacks, Metric
from torchsystem.metrics.average import Loss, Accuracy

classifier = compiler(model, criterion, optimizer)
callbacks = Callbacks(Loss(), Accuracy())

with Session(classifiers) as session:
    session.on(StopIteration)(lambda: session.commit())
    classifiers.put(classifier)
    for epoch in range(5):
        for phase, loader in loaders:
            classifier.phase = phase
            classifier.fit(loader, callbacks) if phase == 'train' else classifier.evaluate(loader, callbacks)
            
            for callback in callbacks:
                classifier.deliver(Metric(callback.name, callback.average.value, phase, epoch), 'results')
                callback.reset()

INFO:__main__:Building classifier with:
INFO:__main__:Model attributes:
INFO:__main__:Metadata(type='model', hash='8ebb966c654c0489e8c32d2ed4d0a688', name='MLP', arguments={'input_size': 784, 'hidden_size': 128, 'output_size': 10, 'p': 0.5, 'activation': 'relu'})
INFO:__main__:Criterion attributes:
INFO:__main__:Metadata(type='criterion', hash='422b735afaea4d9a688f99fbfa3c6de9', name='CrossEntropyLoss', arguments={})
INFO:__main__:Optimizer attributes:
INFO:__main__:Metadata(type='optimizer', hash='55b114a06ec82cec384573d24663dafe', name='Adam', arguments={'lr': 0.001})
INFO:__main__:Compiled classifier on device: cuda
INFO:examples._2_with_events.handlers:.:Batch: 200 loss: 1.064, accuracy: 0.648
INFO:examples._2_with_events.handlers:.:Batch: 400 loss: 0.806, accuracy: 0.740
INFO:examples._2_with_events.handlers:.:Batch: 600 loss: 0.694, accuracy: 0.781
INFO:examples._2_with_events.handlers:.:Batch: 800 loss: 0.630, accuracy: 0.802
INFO:examples._2_with_events.handlers:.:Batch: 1000 l

In [7]:
print(classifier.producer.consumers[0].handlers)

{'Classifier': [<function handle_stored at 0x7f16b6884fe0>, <function handle_restored at 0x7f16b6885580>]}
